In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np
import transformers
import torch
import pandas as pd
import tensorflow as tf
from datasets import load_dataset

print(transformers.__version__)
print(tf.__version__) 



4.11.3
2.6.0


In [2]:
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
import os

if 'COLAB_TPU_ADDR' in os.environ:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu='grpc://' + os.environ['COLAB_TPU_ADDR']
    )
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU initialized.")
else:
    print("TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.")

TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.


**데이타**

In [4]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'aug_csv')
train_data_path = os.path.join(data_dir, 'aug_plus_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...


In [5]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 14281
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14281 entries, 0 to 14280
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           14281 non-null  int64 
 1   class         14281 non-null  object
 2   conversation  14281 non-null  object
dtypes: int64(1), object(2)
memory usage: 334.8+ KB


In [6]:
# 클래스 라벨을 숫자로 변환
# 0, 1, 2, 3, 4
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

class_dict = {}
for idx, class_name in enumerate(CLASS_NAMES):
    class_dict[class_name] = idx

train_data['label'] = train_data['class'].apply(lambda s: class_dict[s])

train_data.head()

,idx,class,conversation,label
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...,3
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...,3
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...,4
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...,2
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...,3


In [7]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [8]:
train_data["conversation"] = [preprocess_sentence(s) for s in train_data["conversation"]]
train_data.head()

,idx,class,conversation,label
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...,3
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...,3
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...,4
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...,2
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...,3


In [9]:
# train/test 분리
df = train_data

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['conversation'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label']
)

**test data**

In [1]:
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
test_data_path = os.path.join(data_dir, 'test.csv')


test_data = pd.read_csv(test_data_path)
test_data.head()

NameError: name 'os' is not defined

**beomi/kcbert-base**

In [10]:
from transformers import TFBertForSequenceClassification

model_name = "beomi/kcbert-base"

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained(model_name)

#PyTorch로 학습된 beomi/kcbert-base 모델을 TensorFlow로 변환하여 사용
from transformers import TFBertModel

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_dict),  from_pt=True)#  클래스 수에 맞게

# tokenizer의 vocab 사이즈 확인
tokenizer.vocab_size


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30000

In [11]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)



In [12]:
# TensorFlow 데이터셋으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 배치 처리 및 셔플
train_dataset = train_dataset.shuffle(1000).batch(16)
val_dataset = val_dataset.shuffle(1000).batch(16)

In [14]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_name)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [15]:
# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

#model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])


In [16]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108918528 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 108,922,373
Trainable params: 108,922,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)

Epoch 1/3
 17/714 [..............................] - ETA: 3:36:10 - loss: 1.4832 - accuracy: 0.3787

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


In [ ]:
# 예측 함수
test_encodings = tokenizer(test_data, truncation=True, padding=True, max_length=128)

test_dataset = tf.data.Dataset.from_tensor_slices(dict(text_encodings)).batch(16)

logits = model.predict(test_dataset).logits

predictions = np.argmax(logits, axis=-1)

print(predictions)  # 예측된 클래스 인덱스 출력